In [1]:
%matplotlib inline

from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input, Activation, merge, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import History, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
import keras.preprocessing.image as image
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.applications.resnet50 import identity_block, conv_block

import numpy as np

import matplotlib.pyplot as plt

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
ResNet = load_model('/home/ubuntu/INV-RES.model')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [3]:
for layer in ResNet.layers[:-7]: layer.trainable = False

ResNet.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

ResNet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 3, 224, 224)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 230, 230)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 112, 112)  9472        zeropadding2d_1[0][0]            
___________________________________________________________________________________________

In [4]:
batch_size = 32

train_gen = image.ImageDataGenerator(horizontal_flip = True,
                               rotation_range = 30,
                               shear_range = 0.2,
                               zoom_range = 0.2,
                                height_shift_range = 0.1,
                                width_shift_range = 0.1, 
                               #preprocessing_function=preprocess_input
                                )


batches = train_gen.flow_from_directory("data/invasive/train", 
                                        target_size = (224, 224), 
                                        batch_size = batch_size, 
                                        )

valid_gen = image.ImageDataGenerator(#preprocessing_function = preprocess_input
                                    )

valid_batches = valid_gen.flow_from_directory("data/invasive/valid", 
                                              target_size = (224, 224), 
                                              batch_size = batch_size, 
                                              shuffle = False, 
                                             )

Found 2095 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [6]:
ResHistory = History()

ResHistory = ResNet.fit_generator(generator =batches, 
                                  samples_per_epoch=batches.nb_sample, 
                                  nb_epoch = 15, 
                                  validation_data=valid_batches, 
                                  nb_val_samples=valid_batches.nb_sample, 
                                  callbacks = ([ResHistory,
                                                EarlyStopping(monitor='val_acc', patience = 3),
                                                ModelCheckpoint('/home/ubuntu/INV-RES2.model',
                                                                monitor='val_acc', 
                                                                save_best_only=True)
                                               ]))

Epoch 1/15
2095/2095 [==============================] - 88s - loss: 0.2729 - acc: 0.8854 - val_loss: 0.3060 - val_acc: 0.8950
Epoch 2/15
2095/2095 [==============================] - 80s - loss: 0.2307 - acc: 0.9036 - val_loss: 0.2239 - val_acc: 0.9100
Epoch 3/15
2095/2095 [==============================] - 79s - loss: 0.2057 - acc: 0.9150 - val_loss: 0.2132 - val_acc: 0.9150
Epoch 4/15
2095/2095 [==============================] - 78s - loss: 0.1794 - acc: 0.9265 - val_loss: 0.1990 - val_acc: 0.9400
Epoch 5/15
2095/2095 [==============================] - 80s - loss: 0.1674 - acc: 0.9303 - val_loss: 0.2976 - val_acc: 0.9050
Epoch 6/15
2095/2095 [==============================] - 78s - loss: 0.1446 - acc: 0.9422 - val_loss: 0.1765 - val_acc: 0.9450
Epoch 7/15
2095/2095 [==============================] - 77s - loss: 0.1621 - acc: 0.9337 - val_loss: 0.1808 - val_acc: 0.9350
Epoch 8/15
2095/2095 [==============================] - 78s - loss: 0.1234 - acc: 0.9489 - val_loss: 0.1863 - val_acc:

In [9]:
ResNet.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
ResHistory2 = History()

ResHistory2 = ResNet.fit_generator(generator =batches, 
                                  samples_per_epoch=batches.nb_sample, 
                                  nb_epoch = 10, 
                                  validation_data=valid_batches, 
                                  nb_val_samples=valid_batches.nb_sample, 
                                  callbacks = ([ResHistory2,
                                                #EarlyStopping(monitor='val_acc', patience = 3),
                                                ModelCheckpoint('/home/ubuntu/INV-RES2.model',
                                                                monitor='val_acc', 
                                                                save_best_only=True)
                                               ]))

Epoch 1/10
2095/2095 [==============================] - 87s - loss: 0.1442 - acc: 0.9399 - val_loss: 0.1875 - val_acc: 0.9450
Epoch 2/10
2095/2095 [==============================] - 77s - loss: 0.1314 - acc: 0.9427 - val_loss: 0.2270 - val_acc: 0.9350
Epoch 3/10
2095/2095 [==============================] - 79s - loss: 0.1337 - acc: 0.9442 - val_loss: 0.1748 - val_acc: 0.9600
Epoch 4/10
2095/2095 [==============================] - 77s - loss: 0.1387 - acc: 0.9451 - val_loss: 0.2111 - val_acc: 0.9500
Epoch 5/10
2095/2095 [==============================] - 77s - loss: 0.1191 - acc: 0.9537 - val_loss: 0.1694 - val_acc: 0.9500
Epoch 6/10
2095/2095 [==============================] - 79s - loss: 0.1340 - acc: 0.9480 - val_loss: 0.1648 - val_acc: 0.9450
Epoch 7/10
2095/2095 [==============================] - 78s - loss: 0.1030 - acc: 0.9628 - val_loss: 0.1661 - val_acc: 0.9450
Epoch 8/10
2095/2095 [==============================] - 80s - loss: 0.1092 - acc: 0.9561 - val_loss: 0.2083 - val_acc:

In [11]:
ResNet.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
ResHistory3 = History()

ResHistory3 = ResNet.fit_generator(generator =batches, 
                                  samples_per_epoch=batches.nb_sample, 
                                  nb_epoch = 10, 
                                  validation_data=valid_batches, 
                                  nb_val_samples=valid_batches.nb_sample, 
                                  callbacks = ([ResHistory3,
                                                #EarlyStopping(monitor='val_acc', patience = 3),
                                                ModelCheckpoint('/home/ubuntu/INV-RES2.model',
                                                                monitor='val_acc', 
                                                                save_best_only=True)
                                               ]))

Epoch 1/10
2095/2095 [==============================] - 90s - loss: 0.0967 - acc: 0.9618 - val_loss: 0.1582 - val_acc: 0.9600
Epoch 2/10
2095/2095 [==============================] - 80s - loss: 0.0872 - acc: 0.9704 - val_loss: 0.1494 - val_acc: 0.9650
Epoch 3/10
2095/2095 [==============================] - 80s - loss: 0.0977 - acc: 0.9604 - val_loss: 0.1475 - val_acc: 0.9650
Epoch 4/10
2095/2095 [==============================] - 79s - loss: 0.0919 - acc: 0.9695 - val_loss: 0.1409 - val_acc: 0.9650
Epoch 5/10
2095/2095 [==============================] - 79s - loss: 0.0909 - acc: 0.9666 - val_loss: 0.1423 - val_acc: 0.9650
Epoch 6/10
2095/2095 [==============================] - 79s - loss: 0.0832 - acc: 0.9685 - val_loss: 0.1489 - val_acc: 0.9700
Epoch 7/10
2095/2095 [==============================] - 82s - loss: 0.0858 - acc: 0.9666 - val_loss: 0.1441 - val_acc: 0.9650
Epoch 8/10
2095/2095 [==============================] - 80s - loss: 0.0799 - acc: 0.9733 - val_loss: 0.1539 - val_acc:

In [13]:
ResHistory4 = History()

ResHistory4 = ResNet.fit_generator(generator =batches, 
                                  samples_per_epoch=batches.nb_sample, 
                                  nb_epoch = 10, 
                                  validation_data=valid_batches, 
                                  nb_val_samples=valid_batches.nb_sample, 
                                  callbacks = ([ResHistory4,
                                                #EarlyStopping(monitor='val_acc', patience = 3),
                                                ModelCheckpoint('/home/ubuntu/INV-RES2.model',
                                                                monitor='val_acc', 
                                                                save_best_only=True)
                                               ]))

Epoch 1/10
2095/2095 [==============================] - 85s - loss: 0.0827 - acc: 0.9656 - val_loss: 0.1663 - val_acc: 0.9600
Epoch 2/10
2095/2095 [==============================] - 77s - loss: 0.0806 - acc: 0.9709 - val_loss: 0.1494 - val_acc: 0.9550
Epoch 3/10
2095/2095 [==============================] - 78s - loss: 0.0742 - acc: 0.9723 - val_loss: 0.1415 - val_acc: 0.9650
Epoch 4/10
2095/2095 [==============================] - 78s - loss: 0.0727 - acc: 0.9728 - val_loss: 0.1608 - val_acc: 0.9600
Epoch 5/10
2095/2095 [==============================] - 77s - loss: 0.0691 - acc: 0.9761 - val_loss: 0.1438 - val_acc: 0.9600
Epoch 6/10
2095/2095 [==============================] - 76s - loss: 0.0748 - acc: 0.9718 - val_loss: 0.1433 - val_acc: 0.9650
Epoch 7/10
2095/2095 [==============================] - 77s - loss: 0.0836 - acc: 0.9666 - val_loss: 0.1580 - val_acc: 0.9650
Epoch 8/10
2095/2095 [==============================] - 76s - loss: 0.0682 - acc: 0.9766 - val_loss: 0.1434 - val_acc:

In [15]:
acc = ResHistory.history['acc'] +  ResHistory2.history['acc'] + ResHistory3.history['acc'] +  ResHistory.history['acc']